# 2. AUTOMATICALLY FINDING LOCATOR

In [1]:
# import argparse

# parser = argparse.ArgumentParser(description='Arguments information.')

# parser.add_argument('-t', '--test-case-path', dest='test_case_path', type=str,
#                     help='A required test case file path. (TXT format)', default='test_cases.pkl')
# parser.add_argument('-c', '--config-path', dest='config_path', type=str,
#                     help='A required configuration file path. (TXT format)', default='config.ini')
# parser.add_argument('-d', '--dictionary-path', dest='dictionary_path', type=str,
#                     help='A required dictionary file path. (JSON format', default='dictionary.json')
# parser.add_argument('-o', '--output-path', dest='output_path', type=str,
#                     help='A required output file path.', default='output.json')
# parser.add_argument('-p', '--proxy', dest='proxy', type=str,
#                     help='An optional proxy.', default='10.10.10.10:8080')

In [2]:
# args = parser.parse_args()
# test_case_path = args.test_case_path
# config_path = args.config_path
# dictionary_path = args.dictionary_path
# output_path = args.output_path
# proxy = args.proxy

In [3]:
test_case_path = 'test_cases.pkl'
config_path = 'config.ini'
dictionary_path = 'dictionary.json'
output_path = 'output_scenario.json'
#proxy = "10.10.10.10:8080"

## 2.1 Reading test cases from serialized file

In [4]:
import pickle
from scenario import *
import json

In [5]:
with open(test_case_path, 'rb') as f:
    test_cases = pickle.load(f)

In [6]:
from selenium import webdriver
from selenium.webdriver.common.proxy import *

## 2.2 Reading configuration

In [7]:
import ConfigParser

In [8]:
class JSON_Writer():
    def __init__(self, filename):
        self.dct = {'browser': 'Chrome',}
        self.hfile = open(filename, 'wb');
#     def __enter__(self):
#         return self;
    
#     def __exit__(self):
#         pass
#         self.hfile.close();
        
    def Add_data(self, key, value):
        self.dct[key] = value;
        
    def Dump(self):
        json.dump(self.dct, self.hfile, indent = 4)
        
    def close(self):
        self.hfile.close();


class Config_parser:
    def __init__(self, fileconfig):
        self.config = ConfigParser.ConfigParser();
        self.config.read(fileconfig);
    
    def get(self, session, key):
        return self.config.get(session, key);

In [9]:
# Reading JSON file
def JSON_reader(filename):
    with open(filename) as data_file:
        data = json.load(data_file)
    return data

config = Config_parser(config_path);
link = config.get('Setting', 'URL');

In [10]:
#PROXY = "10.10.10.10:8080" # IP:PORT or HOST:PORT

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)

# driver = webdriver.Chrome(chrome_options=chrome_options)
# driver.get(link)

## 2.3 Defining dictionary for addition information according to frame keys 

In [11]:
with open(dictionary_path, 'r') as f:
    dic = f.read()
    dictionary = json.loads(dic)

In [12]:
action_dict = {"write": ['field', 'box'], 
              "click": [],
              "select": []}
obj_mod_dict = {"search": ['search']}
obj_dict = {"button": ['button'], "item": ['title', 'h2', 'h3'], "quantity": ["quantity", "qty"],
           "box": ['box', 'text']}

In [13]:
from fuzzywuzzy import fuzz # Library for fuzzy matching
from difflib import SequenceMatcher

## 2.4 Finding locator

In [14]:
import time

In [15]:

test_case_lst = []
tsc_idx = 0
print "Number of test cases: ", len(test_cases)
for test_case in test_cases:
    driver = webdriver.Chrome("d:\\Project\\AI\\automation\\AI_Selenium\\bin\\chromedriver\\chromedriver.exe",chrome_options=chrome_options)
    #driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(link)
    test_step = [];
    print "Number of test steps: ", len(test_case)
    stn_idx = 0
    while stn_idx < len(test_case):
        sentence = test_case[stn_idx]
#     for sentence in test_case:
        driver.execute_script("""
        window.getPathTo = function (element) {
            if (element.id!=='')
                return "//*[@id='"+element.id+"']";
            if (element===document.body)
                return element.tagName;

            var ix= 0;
            var siblings= element.parentNode.childNodes;
            for (var i= 0; i<siblings.length; i++) {
                var sibling= siblings[i];
                if (sibling===element)
                    return window.getPathTo(element.parentNode)+'/'+element.tagName+'['+(ix+1)+']';
                if (sibling.nodeType===1 && sibling.tagName===element.tagName)
                    ix++;
            }
        }
        """)
        print "*"*20
        print "Test step: ", sentence.text
        print "Action: ", sentence.frames[0].action[0]
#         print "Modifier of actor: ", sentence.frames[0].actor_mod
        print "Object modifier: ", sentence.frames[0].obj_mod
        print "Object: ", sentence.frames[0].obj[0]
        print "Expectation: ", sentence.expectation
        frame = sentence.frames[0]
        
        if frame.obj[0].lower() in dictionary:
            next_step = test_case[stn_idx + 1]
            print next_step.frames[0].obj_mod
            if next_step.frames[0].obj_mod == "positive":
                if int(sentence.expectation) < 0:
                    sentence.expectation = str(int(sentence.expectation) * -1)
            elif next_step.frames[0].obj_mod == "negative":
                if int(sentence.expectation) > 0:
                    sentence.expectation = str(int(sentence.expectation) * -1)
            stn_idx += 1
        
        xpath = '';
        thirdpara = sentence.expectation;
        # Write Text action
        driver.implicitly_wait(5)
        print "Running..."
        if SequenceMatcher(None, frame.action[0], "write").ratio() > 0.75:
#             elements = driver.find_elements_by_xpath('.//body//*[contains(@type, "text")]')
            elements = driver.find_elements_by_xpath('.//body//*[@type="text"]')
            elements += driver.find_elements_by_xpath('.//body//*[@type="search"]')
            print "Number of elements that need to check: ", len(elements)
            if len(elements) > 0:
                weights = [0.] * len(elements)
                max_weights = 0.0
                suitable_idx = 0
                # Update weights
                for idx in range(0, len(elements)):
                    element = elements[idx]
                    if fuzz.partial_token_set_ratio(frame.obj_mod, "search") > 75:
                        if element.size['width'] > 500 or element.size['width'] < 40:
                            continue
                        if element.size['height'] > 100 or element.size['height'] < 15:
                            continue
                    
                    if element.tag_name == "script" or element.tag_name == "style":
                        continue
                    current_parent = element.find_elements_by_xpath('..')
                    depth = 1 # Number of parents that we want to trace back
                    while len(current_parent) > 0:
                        depth -= 1
                        if depth == 0:
                            break
                        if current_parent[0].tag_name == 'html':
                            break
                        if current_parent[0].tag_name == "form":
                            weights[idx] += 10
                        current_parent = current_parent[0].find_elements_by_xpath('..')
                    
#                     current_parent = element.find_elements_by_xpath('..')
#                     print "Current parent: ", len(current_parent)
#                     depth = 1 # Number of parents that we want to trace back
#                     while len(current_parent) > 0:
#                         depth -= 1
#                         if depth == 0:
#                             break
#                         if current_parent[0].tag_name == 'html':
#                             break
#                         current_parent = current_parent[0].find_elements_by_xpath('..')
                    
                    if len(current_parent) > 0:
                        current_element = current_parent[0]
                    if current_element.get_attribute("disabled") == "disabled":
                        continue
                    outerHTML = current_element.get_attribute('outerHTML')
                    # Leverage information from test step sentence to find exactly XPath
                    for keyword in action_dict["write"]:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1./100
                    for keyword in obj_dict[frame.obj[0]]:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1./100
                    try:
                        for keyword in obj_mod_dict[frame.obj_mod]:
                            weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1. / 60
                    except:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, frame.obj_mod)*1. / 60
                    if weights[idx] > max_weights:
                        max_weights = weights[idx]
                        suitable_idx = idx
                suitable_element = elements[suitable_idx]
                print "Element found: ", suitable_element.get_attribute('outerHTML')
                xpath = driver.execute_script("return window.getPathTo(arguments[0]);", suitable_element)
                print "XPath found: ", xpath
                suitable_element.clear()
                suitable_element.send_keys(sentence.expectation)
        elif SequenceMatcher(None, frame.action[0], "click").ratio() > 0.75:
            elements = []
            if SequenceMatcher(None, frame.obj[0], "button").ratio() > 0.8: # If object is button
                    elements += driver.find_elements_by_xpath('//body//*[contains(@type, "submit")]')
                    elements += driver.find_elements_by_xpath('//body//*[contains(@type, "button")]')
                    elements += driver.find_elements_by_xpath('//body//*[contains(@role, "button")]')
            elif fuzz.ratio(frame.obj[0], "item") > 0.8: # If object is item
                elements += driver.find_elements_by_xpath('//body//*[@href]')
            else:
                elements += driver.find_elements_by_xpath('//body//*[@href]') # Otherwise
            print "Number of elements that need to check: ", len(elements)
            if len(elements) > 0:
                weights = [0.] * len(elements)
                max_weights = 0.0
                suitable_idx = 0
                for idx in range(0, len(elements)):
                    element = elements[idx]
                    if element.is_displayed() == False:
                        continue
                    current_parent = element.find_elements_by_xpath('..')

                    depth = 1 # Number of parents that we want to trace back
                    while len(current_parent) > 0:
                        depth -= 1
                        if depth == 0:
                            break
                        if current_parent[0].tag_name == 'html':
                            break
                        current_parent = current_parent[0].find_elements_by_xpath('..')
                        
                    if len(current_parent) > 0:
                        current_element = current_parent[0]
                    outerHTML = current_element.get_attribute('outerHTML')
                    # Leverage information from test step sentence to find exactly XPath
                    for keyword in action_dict["click"]:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1. / 100
                    for keyword in obj_dict[frame.obj[0]]:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1. / 100
                    try:
                        for keyword in obj_mod_dict[frame.obj_mod]:
                            weights[idx] += fuzz.partial_token_set_ratio(outerHTML, keyword)*1. / 100
                    except:
                        weights[idx] += fuzz.partial_token_sort_ratio(outerHTML, frame.obj_mod)*1. / 60
                        weights[idx] += fuzz.partial_token_sort_ratio(current_element.text, frame.obj_mod)*1. / 60
                    if fuzz.ratio(frame.obj[0], "item") > 0.8 and len(sentence.expectation) > 0:
                        weights[idx] += fuzz.partial_token_set_ratio(outerHTML, sentence.expectation)
                    if weights[idx] > max_weights:
                        max_weights = weights[idx]
                        suitable_idx = idx
                suitable_element = elements[suitable_idx]
                print "Element found: ", suitable_element.get_attribute('outerHTML')
                xpath = driver.execute_script("return window.getPathTo(arguments[0]);", suitable_element)
                print "XPath found: ", xpath
                suitable_element.click()
        elif SequenceMatcher(None, frame.action[0], "select").ratio() > 0.75:
            elements = []
            elements += driver.find_elements_by_xpath('//body//select')
            print "Number of elements that need to check: ", len(elements)
            if len(elements) > 0:
                weights = [0.] * len(elements)
                max_weights = 0.0
                suitable_idx = 0
                for idx in range(0, len(elements)):
                    element = elements[idx]
                    if element.is_displayed() == False:
                        continue
                    current_parent = element.find_elements_by_xpath('..')

                    depth = 1 # Number of parents that we want to trace back
                    while len(current_parent) > 0:
                        depth -= 1
                        if depth == 0:
                            break
                        if current_parent[0].tag_name == 'html':
                            break
                        current_parent = current_parent[0].find_elements_by_xpath('..')
                        
                    if len(current_parent) > 0:
                        current_element = current_parent[0]
                    outerHTML = current_element.get_attribute('outerHTML')
                    if len(outerHTML) < 100:
                        weights[idx] += 20
                    # Leverage information from test step sentence to find exactly XPath
                    for keyword in action_dict["select"]:
                        weights[idx] += fuzz.partial_token_sort_ratio(outerHTML, keyword)*1. / 100
                    weights[idx] += fuzz.partial_token_sort_ratio(outerHTML, sentence.frames[0].obj[0])*1. /20
                    for keyword in obj_dict[frame.obj[0]]:
                        weights[idx] += fuzz.partial_token_sort_ratio(outerHTML, keyword)
                    try:
                        for keyword in obj_mod_dict[frame.obj_mod]:
                            weights[idx] += fuzz.partial_token_sort_ratio(outerHTML, keyword)
                    except:
                        pass
                    if weights[idx] > max_weights:
                        max_weights = weights[idx]
                        suitable_idx = idx
                suitable_element = elements[suitable_idx]
                print "Element found: ", suitable_element.get_attribute('outerHTML')
                xpath = driver.execute_script("return window.getPathTo(arguments[0]);", suitable_element)
                print "XPath found: ", xpath
                selecting_elements = driver.find_elements_by_xpath(xpath + "/option[text()="+ sentence.expectation + "]")
                if len(selecting_elements) > 0:
                    selecting_elements[0].click()
            
        # Write to JSON        
        modified_action = sentence.frames[0].action[0]
        if SequenceMatcher(None, modified_action, 'select').ratio() >= 0.8:
            modified_action = "SelectOption"
        test_step.append(
        {
            'name': sentence.text,
            'action': modified_action,
            'locateElement': {'by': 'Xpath', 'value': xpath},
            'thirdPara': thirdpara,
        })
        stn_idx += 1
        print ""
    time.sleep(5)
    driver.close()
    test_case_lst += test_step;
    tsc_idx += 1
print ""
print "*"*20
print "Done!"
print "*"*20
# test_suite = {
#     'name': str(tsc_idx),
#     'testcase': test_case_lst,}
# test_case_container = 
test_suite = {
    'name': str(tsc_idx),
    'step': test_case_lst,}


f = JSON_Writer(output_path);
f.Add_data('testsuite', test_suite);
f.Dump();
f.close();

Number of test cases:  2
Number of test steps:  6
********************
Test step:  User writes search box
Action:  writes
Object modifier:  search
Object:  box
Expectation:  Data Mining
Running...
Number of elements that need to check:  1
Element found:  <input type="text" class="gh-tb ui-autocomplete-input" size="50" maxlength="300" placeholder="Search for anything" id="gh-ac" name="_nkw" autocapitalize="off" autocorrect="off" spellcheck="false" autocomplete="off">
XPath found:  //*[@id='gh-ac']

********************
Test step:  User clicks search button
Action:  clicks
Object modifier:  search
Object:  button
Expectation:  
Running...
Number of elements that need to check:  4
Element found:  <input type="submit" class="btn btn-prim gh-spr" id="gh-btn" value="Search">
XPath found:  //*[@id='gh-btn']

********************
Test step:  User clicks an item
Action:  clicks
Object modifier:  None
Object:  item
Expectation:  Data Mining
Running...
Number of elements that need to check:  301
